In [1]:
print("Start")

import pandas as pd

data = pd.read_csv("../../../dataset-sis-energy-derived-reanalysis-temperature-country_daily/H_ERA5_ECMW_T639_TA-_0002m_Euro_NUT2_S197901010000_E202309302300_INS_TIM_01d_subcountry_2023_10_15.csv", sep=";")

Start


In [2]:
# data.head()
# data.info()

In [3]:
df = data[["Date", "IS00"]].copy()
df

,Date,IS00
0,1979-01-01,262.266990
1,1979-01-02,260.346808
2,1979-01-03,259.511792
3,1979-01-04,258.582429
4,1979-01-05,268.091415
...,...,...
16339,2023-09-26,278.479792
16340,2023-09-27,277.396087
16341,2023-09-28,276.261230
16342,2023-09-29,275.364654


In [4]:
# convert column holding the date from Object to Datetime
df["Date"] = pd.to_datetime(df["Date"])

# resample from daily to week data and aggregate on mean temperature values over the week
# not resample("7D") as before as overflow to following year happens
# df = df.resample("1W", on="Date").max()

# convert temperature values from Kelvin to Celsius
df["IS00"] = df["IS00"] - 273.15

In [5]:
# reset index from date value to integer 
df = df.reset_index()
# get weeknumber as separate column
df["weeknumber"] = df.Date.dt.isocalendar().week
# get year number as separate column
df["year"] = df.Date.dt.isocalendar().year

In [6]:
# view data
# df

In [7]:
# set to "hot", when temp higher than 20 degrees
compareToInCelsius = 20
df["IS00_masked"] = df.IS00.mask(df.IS00 > compareToInCelsius, "hot")

In [8]:
grouped = df.groupby(["year", "weeknumber"])
grouped.describe()

index                                                 \
                count     mean       std      min       25%      50%   
year weeknumber                                                        
1979 1            7.0      3.0  2.160247      0.0      1.50      3.0   
     2            7.0     10.0  2.160247      7.0      8.50     10.0   
     3            7.0     17.0  2.160247     14.0     15.50     17.0   
     4            7.0     24.0  2.160247     21.0     22.50     24.0   
     5            7.0     31.0  2.160247     28.0     29.50     31.0   
...               ...      ...       ...      ...       ...      ...   
2023 35           7.0  16313.0  2.160247  16310.0  16311.50  16313.0   
     36           7.0  16320.0  2.160247  16317.0  16318.50  16320.0   
     37           7.0  16327.0  2.160247  16324.0  16325.50  16327.0   
     38           7.0  16334.0  2.160247  16331.0  16332.50  16334.0   
     39           6.0  16340.5  1.870829  16338.0  16339.25  16340.5   

                                    IS00             ...             \
                      75%      max count       mean  ...        75%   
year weeknumber                                      ...              
1979 1               4.50      6.0   7.0 -10.878992  ...  -9.601188   
     2              11.50     13.0   7.0 -12.443719  ... -11.916554   
     3              18.50     20.0   7.0  -5.078189  ...  -3.704522   
     4              25.50     27.0   7.0  -9.739560  ...  -8.409665   
     5              32.50     34.0   7.0 -13.354690  ... -10.938186   
...                   ...      ...   ...        ...  ...        ...   
2023 35          16314.50  16316.0   7.0   7.605134  ...   8.036499   
     36          16321.50  16323.0   7.0   6.896728  ...   8.350603   
     37          16328.50  16330.0   7.0   4.011067  ...   4.855609   
     38          16335.50  16337.0   7.0   3.031933  ...   4.428283   
     39          16341.75  16343.0   6.0   3.827251  ...   5.058866   

                           IS00_masked                                  \
                       max       count       mean       std        min   
year weeknumber                                                          
1979 1           -5.058585         7.0 -10.878992  3.282839 -14.567571   
     2           -4.261540         7.0 -12.443719  4.282837 -18.267270   
     3           -1.714672         7.0  -5.078189  2.430303  -8.567298   
     4           -6.126030         7.0  -9.739560  2.430776 -12.640590   
     5          -10.516842         7.0 -13.354690  3.029798 -17.720412   
...                    ...         ...        ...       ...        ...   
2023 35           9.342903         7.0   7.605134  1.079680   5.790613   
     36           9.219127         7.0   6.896728  2.164586   3.160001   
     37           6.540375         7.0   4.011067  1.611974   2.208161   
     38           5.281538         7.0   3.031933  1.871889   0.379566   
     39           5.584088         6.0   3.827251  1.445966   2.214654   

                                                             
                       25%        50%        75%        max  
year weeknumber                                              
1979 1          -13.220700 -10.883010  -9.601188  -5.058585  
     2          -14.391551 -11.961015 -11.916554  -4.261540  
     3           -6.550783  -4.754745  -3.704522  -1.714672  
     4          -11.874588  -8.841795  -8.409665  -6.126030  
     5          -15.888192 -11.592817 -10.938186 -10.516842  
...                    ...        ...        ...        ...  
2023 35           7.263723   7.501977   8.036499   9.342903  
     36           5.679716   7.837334   8.350603   9.219127  
     37           2.907101   3.803510   4.855609   6.540375  
     38           1.611508   3.482845   4.428283   5.281538  
     39           2.636050   3.678659   5.058866   5.584088  

[2335 rows x 24 columns]

In [9]:
df_grouped = grouped["IS00"].max().to_frame(name="max_daily_in_week").reset_index()
df_grouped

,year,weeknumber,max_daily_in_week
0,1979,1,-5.058585
1,1979,2,-4.261540
2,1979,3,-1.714672
3,1979,4,-6.126030
4,1979,5,-10.516842
...,...,...,...
2330,2023,35,9.342903
2331,2023,36,9.219127
2332,2023,37,6.540375
2333,2023,38,5.281538


In [10]:
# save result to a csv file
df_grouped.to_json('temp_subcountry_IS00_weekly_max_daily_in_week.json', orient='records')
